In [11]:
import requests
import pandas as pd
import datetime
import time

keys = ['XCMSFDTYUZGLCJBS','4R5NDTM0D1HKQA9V','MY2FVAV6O66MO17X']

In [20]:
def DOW_analysis(symbols):
    final_df = pd.DataFrame()
    final_weighted_df = pd.DataFrame()
    failed_symbols = []
    for index,symbol in enumerate(symbols):
        key = keys[index%len(keys)]
        try:
            print(f'Reading in {symbol}')
            url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={key}'
            r = requests.get(url)
            data = r.json()

            df = pd.DataFrame.from_dict(data['Time Series (Daily)']).transpose()
            df= df.astype('float')
            df['Date'] = df.apply(lambda row: datetime.datetime.strptime(row.name, '%Y-%m-%d'),axis=1)
            df['Day'] = df.apply(lambda row: row['Date'].strftime('%A'),axis=1)
            df = df.reset_index(drop=True)
            df['Delta'] = df.apply(lambda row: row['4. close']-row['1. open'],axis=1)
            df['Delta %'] = df.apply(lambda row: row['Delta']/row['1. open']*100,axis=1)

            df['Time Delta'] = df.apply(lambda row: (df.loc[0,'Date']-row['Date']).days,axis=1)
            df['Time Delta'] = df.iloc[-1,9]-df['Time Delta']
#             df['DeltaW'] = df.apply(lambda row: row['4. close']-row['1. open'],axis=1)

            df['DeltaW %'] = df.apply(lambda row: row['Delta']/row['1. open']*100,axis=1)
            df['DeltaW %'] = df['DeltaW %']*df['Time Delta']

            agg_df = df.groupby('Day').mean()['Delta %']
            agg_df2 = df.groupby('Day').sum()[['Time Delta','DeltaW %']]
            agg_df2 = agg_df2['DeltaW %']/agg_df2['Time Delta']

            final_df = final_df.append(agg_df,ignore_index=True)
            final_weighted_df = final_weighted_df.append(agg_df2,ignore_index=True)
            
        except Exception as e:
            print(f'Failed to process {symbol}')
            print(e)
            failed_symbols.append(symbol)
        
        time.sleep(3)
        
    symbols = [x for x in symbols if x not in failed_symbols]
    final_df['Symbols'] = symbols
    final_weighted_df['Symbols'] = symbols
    
    final_df = final_df[['Symbols','Monday','Tuesday','Wednesday','Thursday','Friday']]
    final_weighted_df = final_weighted_df[['Symbols','Monday','Tuesday','Wednesday','Thursday','Friday']]
    
    return final_df, final_weighted_df

In [21]:
symbols = ['AMZN', 'NFLX', 'BABA', 'SPOT', 'JD', 'TSLA', 'SBUX', 'FB', 'AAPL', 'GSK', 'UA', 'NTDOY','PM', 'GOOG', 'TTWO', 'EA', 'ATVI']
symbols = ['TSLA','SBUX']

In [25]:
df3,df4 = DOW_analysis(symbols)

Reading in TSLA
Reading in SBUX


In [26]:
df3

,Symbols,Monday,Tuesday,Wednesday,Thursday,Friday
0,TSLA,0.332423,0.107302,0.000407,-0.061355,-0.161222
1,SBUX,0.017235,0.073359,0.062815,0.144053,0.016721


In [27]:
df4

,Symbols,Monday,Tuesday,Wednesday,Thursday,Friday
0,TSLA,0.560769,0.038871,0.028535,-0.034076,-0.139993
1,SBUX,-0.015010,0.052256,0.042557,0.115268,0.065590


In [23]:
df1

,Symbols,Monday,Tuesday,Wednesday,Thursday,Friday
0,AMZN,0.082954,0.107494,0.039594,0.077859,-0.028569
1,NFLX,0.122490,0.064859,0.107498,0.296126,-0.144407
2,BABA,-0.020183,0.045027,0.023804,-0.206320,-0.200342
3,SPOT,0.117965,-0.010704,-0.041751,0.295095,-0.040117
4,JD,-0.119900,0.087875,0.007166,-0.150786,-0.048688
5,AAPL,0.212289,-0.029424,0.066140,0.015467,-0.088869
6,GSK,0.019595,0.052935,0.010298,-0.003509,0.032971
7,UA,-0.037978,0.004653,-0.006092,0.008716,-0.215311
8,NTDOY,0.070851,0.072217,0.102782,-0.000002,0.036711
9,PM,0.087534,0.005949,0.010119,0.112784,0.062916


In [24]:
df2

,Symbols,Monday,Tuesday,Wednesday,Thursday,Friday
0,AMZN,0.068625,0.140725,0.013973,0.105426,-0.094339
1,NFLX,0.083393,0.143879,0.096573,0.167986,-0.055913
2,BABA,0.006342,0.077333,-0.017585,-0.238955,-0.195302
3,SPOT,0.187802,-0.082465,-0.176575,0.286268,-0.032659
4,JD,-0.186591,0.126697,0.045570,-0.076300,-0.080086
5,AAPL,0.246096,0.029142,0.030053,-0.036668,-0.076076
6,GSK,0.001437,0.035873,-0.022712,-0.019779,0.047250
7,UA,-0.017141,0.010178,-0.026454,0.073150,-0.302127
8,NTDOY,0.087864,0.071678,0.122976,0.010125,0.063718
9,PM,0.063083,-0.041662,0.016995,0.117725,0.067341
